# Logistic Regression in PyTorch

In [ ]:
import torch
import torch.nn as nn
from sklearn import datasets                            # to load a binary classification dataset
from sklearn.preprocessing import StandardScaler        # we want to scale our features
from sklearn.model_selection import train_test_split    # separate training and testing data
import matplotlib.pyplot as plt
import numpy

### 0. Preparing the data

In [ ]:
# This is a binary classification problem where we can predict breast cancer from the data.
dataset = datasets.load_breast_cancer()
X, y = dataset.data, dataset.target

n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale our features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Converting to float 32 because np array is originally double.
# Then converting to the torch tensor.
X_train = torch.from_numpy(X_train.astype(numpy.float32))
X_test = torch.from_numpy(X_test.astype(numpy.float32))
y_train = torch.from_numpy(y_train.astype(numpy.float32))
y_test = torch.from_numpy(y_test.astype(numpy.float32))

y_train = y_train.view(y_train.shape[0], 1) # reshape to column vector
y_test = y_test.view(y_test.shape[0], 1)    # reshape to column vector
print(y_train.shape)

torch.Size([455, 1])


### 1. Creating our model
In logistic regression, we have y = wx + b and we apply sigmoid as activation function at the end.

In [ ]:
class LogisticRegression(nn.Module):

  # This is the function where we stack up all our layers.
  # Since logistic regression only has a linear layer, we use that here.
  def __init__(self, input_dim):
    super().__init__()
    # This is the linear layer. This does y = x*W.T + b. The parameters it requires are the input features and the output features.
    # For our dataset, the input features are 30 = input_dim and our output features are 1 i.e if person has cancer or not.
    self.linear = nn.Linear(input_dim, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x)) # Applying the sigmoid function to our model.
    return y_predicted

model = LogisticRegression(n_features)

### 2. Loss and optimizer

In [ ]:
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

### 3. Training Loop

In [ ]:
n_iters = 100

for epoch in range(n_iters):
  # Forward Pass and Loss
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)
  # Backward Pass
  loss.backward()
  # Update
  optimizer.step()
  # Zero Gradients
  optimizer.zero_grad()

  if epoch+1%10 == 0:
    print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

### 4. Evaluation using testing data

In [ ]:
# We dont want this to be a part of the computational graph and calculate the gradients for this operation too.
with torch.no_grad():
  # prediction for test data.
  y_predicted = model(X_test)
  # rounding off predicted y values to 0 or 1 because the output is a probability between 0 and 1.
  # 0 being no cancer and 1 being cancer.
  y_predicted_cls = y_predicted.round()
  # For every prediction that is correct, it will add +1 and then divide it by number of test samples.
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'Accuracy = {acc:.4f}')

Accuracy = 0.9825


We can see that our model has 98.25% accuracy on the dataset.